# DES - Data Encryption Standard

* DES is one of the most widely used encryption schemes
* In this project, we will implement this algorithm in Python
* We create a function that takes a character string of 8 characters or less, encrypts it to cipher text, and then decrypts it back to plaintext
* Any input string entered longer than 8 characters gets truncated

# Phase 1: Generate 16 Round Keys with PC-1 and PC-2

* Our 64 bit key is permuted using the PC-1 table
* Only 56 bits of the original key appear in the permuted key
* We then split this 56 bit key into two halves before performing shifts

In [1]:
PC_1 = [57, 49, 41, 33, 25, 17, 9,
         1, 58, 50, 42, 34, 26, 18,
         10, 2, 59, 51, 43, 35, 27,
         19, 11, 3, 60, 52, 44, 36,
         63, 55, 47, 39, 31, 23, 15,
         7, 62, 54, 46, 38, 30, 22,
         14, 6, 61, 53, 45, 37, 29,
         21, 13, 5, 28, 20, 12, 4]

* We apply the permutation table to each of the concatenated pairs
* The PC-2 table uses 48 of the 56 bits

In [2]:
PC_2 = [14, 17, 11, 24, 1, 5,
         3, 28, 15, 6, 21, 10,
         23, 19, 12, 4, 26, 8,
         16, 7, 27, 20, 13, 2,
         41, 52, 31, 37, 47, 55,
         30, 40, 51, 45, 33, 48,
         44, 49, 39, 56, 34, 53,
         46, 42, 50, 36, 29, 32]

Define the shift schedule

In [3]:
# Number of left shifts for each round
shift_schedule = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

We define functions to perform tasks such as:
* Permutation
* Shifts
* Generation of Round Keys
* Splitting of Keys
* Combining the Split Halves

In [4]:
# Permute bits by table
def permute(bits, table):
  return [bits[i - 1] for i in table]

# Perform a circular shift by n bits
def left_shift(bits, n):
  return bits[n:] + bits[:n]

# Generate 16 round keys and apply PC-1 to get a 56-bit key
def generate_round_keys(key):
  key_56 = permute(key, PC_1)

  # Split the key into two 28-bit halves (H1 and H2)
  H1 = key_56[:28]
  H2 = key_56[28:]

  # Generate 16 round keys
  round_keys = []
  for i in range(16):
    # Perform left circular shifts according to the shift schedule
    H1 = left_shift(H1, shift_schedule[i])
    H2 = left_shift(H2, shift_schedule[i])

    # Combine the two halves and apply PC-2 to get the 48-bit round key
    combined_key = H1 + H2
    round_key = permute(combined_key, PC_2)

    # Add the round key to the list
    round_keys.append(round_key)

  return round_keys

* We define a our hex to binary function, which converts hexadecimal to binary; this is a function we will call several times
* We test by using an example to print out the first 16 subkeys
* We use the hexadecimal key from the slides throughout the entire project

In [5]:
# Convert hexadecimal to 64 bit binary
def hex_to_binary(hex_string):
  binary = ''.join(format(int(char, 16), '04b') for char in hex_string)
  # Ensure the binary representation is exactly 64 bits
  # We pad with zeros if shorter and truncate if longer
  if len(binary) < 64:
    binary = binary.ljust(64, '0')
  elif len(binary) > 64:
    binary = binary[:64]
  return [int(bit) for bit in binary]


# Example using Hello World and hexadecimal key
plain_text = "Hello World"
key_text = "133457799BBCDFF1"

# Convert the input key to a 64-bit binary representation
key_binary = hex_to_binary(key_text)

# Generate 16 round keys
round_keys = generate_round_keys(key_binary)

# Display the round keys
for i, rk in enumerate(round_keys):
  print(f"K{i + 1}: {''.join(map(str, rk))}")

K1: 000110110000001011101111111111000111000001110010
K2: 011110011010111011011001110110111100100111100101
K3: 010101011111110010001010010000101100111110011001
K4: 011100101010110111010110110110110011010100011101
K5: 011111001110110000000111111010110101001110101000
K6: 011000111010010100111110010100000111101100101111
K7: 111011001000010010110111111101100001100010111100
K8: 111101111000101000111010110000010011101111111011
K9: 111000001101101111101011111011011110011110000001
K10: 101100011111001101000111101110100100011001001111
K11: 001000010101111111010011110111101101001110000110
K12: 011101010111000111110101100101000110011111101001
K13: 100101111100010111010001111110101011101001000001
K14: 010111110100001110110111111100101110011100111010
K15: 101111111001000110001101001111010011111100001010
K16: 110010110011110110001011000011100001011111110101


# Phase 2: Process Origin Message Initial Permutation and Split Two Halves (IP Table)

* Next, we are given the initial permutation (IP table)
* We divide this permuted table into left and right halves of 32 bits each

In [6]:
IP = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

In [7]:
# Apply the IP (inital permutation) to the message
def initial_permutation(message):
  return permute(message, IP)

# Apply the IP (inital permutation) to the message
def split_message(permuted_message):
  left_half = permuted_message[:32]
  right_half = permuted_message[32:]
  return left_half, right_half

* We define an important function that converts text to binary
* We then test on an example and print out the left and right halves along with the full permuted message

In [8]:
# Convert plaintext to 64 bit binary representation
def text_to_binary(message):
  binary = ''.join(format(ord(char), '08b') for char in message)
  if len(binary) < 64:
    binary = binary.ljust(64, '0')
  elif len(binary) > 64:
    binary = binary[:64]
  return [int(bit) for bit in binary]

# Example
original_message = "Hello World"

message_binary = text_to_binary(original_message)

# Apply the initial permutation
permuted_message = initial_permutation(message_binary)

# Split the permuted message into two halves
left_half, right_half = split_message(permuted_message)

# Display the results
print(f"Permuted Message: {''.join(map(str, permuted_message))}")
print(f"Left Half: {''.join(map(str, left_half))}")
print(f"Right Half: {''.join(map(str, right_half))}")

Permuted Message: 1101111101000000110111101101001000000000101111101001110111010000
Left Half: 11011111010000001101111011010010
Right Half: 00000000101111101001110111010000


# Phase 3: 16 Round Encryption With Keys (E Bit Selection table, S Boxes, P Box)

* We expand each block from 32 to 48 bits, giving us the E table

In [9]:
E = [32, 1, 2, 3, 4, 5,
     4, 5, 6, 7, 8, 9,
     8, 9, 10, 11, 12, 13,
     12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21,
     20, 21, 22, 23, 24, 25,
     24, 25, 26, 27, 28, 29,
     28, 29, 30, 31, 32, 1]

* We then XOR the output and divide by groups of 6 bits
* This gives us 8 S-boxes

In [10]:
S = [
    # S-boxes

    # S1

     [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
    ],

    # S2

     [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
    ],

    # S3

     [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 2, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
    ],

    # S4

    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
    ],

    # S5

    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
    ],

    # S6

    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
    ],

    # S7

    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 14, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
    ],

    # S8

    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
    ],
]

Permuations are applied to the P table

In [11]:
P = [16, 7, 20, 21,
     29, 12, 28, 17,
     1, 15, 23, 26,
     5, 18, 31, 10,
     2, 8, 24, 14,
     32, 27, 3, 9,
     19, 13, 30, 6,
     22, 11, 4, 25]

In [12]:
# Expand right half from 32 bits to 48 bits
def expand(bits):
  return permute(bits, E)

In [13]:
# Perform XOR between the two lists of bits
def xor(bits1, bits2):
  return [b1 ^ b2 for b1, b2 in zip(bits1, bits2)]

In [14]:
def s_box_substitution(bits):
  # Apply S Box Substitution
  output = []
  for i in range(8):
    # Get the current 6-bit block
    block = bits[i * 6:(i + 1) * 6]
    row = int(f"{block[0]}{block[5]}", 2)  # First and last bits form the row
    col = int(''.join(map(str, block[1:5])), 2)  # Middle 4 bits form the column
    # Use the rows and columns from tables to get the value
    s_box_value = S[i][row][col]
        # Convert the value into a 4-bit binary and append to the output
    output.extend([int(bit) for bit in format(s_box_value, '04b')])
  return output

In [15]:
def p_box_permutation(bits):
  # Applies P permutations
  return permute(bits, P)

* We apply the Feistel Function to combine the last few steps


In [16]:
# The DES round function which combines expansion, XOR, S-box, and P-box
def feistel_function(right_half, round_key):
  # Expansion
  expanded_right = expand(right_half)
  # XOR with the round key
  xored = xor(expanded_right, round_key)
  # S substitution
  substituted = s_box_substitution(xored)
  # P permutation
  permuted = p_box_permutation(substituted)
  return permuted

* We then create our first encryption function which takes the two halves and the round keys as arguments
* The XOR function is performed on the right half, while the left half becomes the old right half

In [17]:
def des_encrypt(left_half, right_half, round_keys):
  # 16 rounds of DES Encryption
  for i in range(16):
    # Save original right half
    original_right = right_half[:]
    # Then apply Feistel function to the right half using the current round key
    right_half = xor(left_half, feistel_function(right_half, round_keys[i]))
    # The new left half becomes the original right half
    left_half = original_right
  return left_half, right_half

We test the encryption algorithm on an example and print the encrypted message in binary

In [18]:
# Implement DES to encrypt the message
def encrypt(message, round_keys):
  # Step 1: Convert the message to a binary
  message_binary = text_to_binary(message)
  # Step 2: Apply initial permutation
  permuted_message = initial_permutation(message_binary)
  # Step 3: Split message into left and right halves
  left_half, right_half = split_message(permuted_message)
  # Step 4: Perform 16 rounds of encryption
  left_half, right_half = des_encrypt(left_half, right_half, round_keys)
  # Step 5: Combine halves and return the result
  return left_half + right_half

# Example
plain_text = "Hello World"
round_keys = generate_round_keys(hex_to_binary("133457799BBCDFF1"))

# Encrypt plaintext
encrypted_message = encrypt(plain_text, round_keys)
print(f"Encrypted Message: {''.join(map(str, encrypted_message))}")

Encrypted Message: 1111100101110001011011111011110110000010110100110100011111000101


# Phase 4: Reverse and Final Permutation (IP^-1 table)

After 16 rounds, reverse the order of the two blocks and apply the final permutation

In [19]:
IP_1 = [40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 43, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25]

In [20]:
# Apply the inverse permutation IP^-1 to the 64 bit
def final_permutation(bits):
  return permute(bits, IP_1)

In [21]:
# Reverse the halves and apply final permutation
# Right half becomes left, left half becomes right
def reverse_and_permute(left_half, right_half):
  combined_bits = right_half + left_half
  # Apply final permutation IP^-1
  final_permuted_bits = final_permutation(combined_bits)
  return final_permuted_bits

Full DES encrytption with the reversed permutations

In [22]:
from binascii import hexlify
def full_des_encryption(message, round_keys):
  # Complete the DES encryption process
  # Convert message to a binary
  message_binary = text_to_binary(message)
  # Step 2: Apply initial permutation
  permuted_message = initial_permutation(message_binary)
  # Step 3: Split message into left and right halves
  left_half, right_half = split_message(permuted_message)
  # Step 4: Perform 16 rounds of encryption
  left_half, right_half = des_encrypt(left_half, right_half, round_keys)
  # Step 5: Reverse halves and apply final permutation
  encrypted_message = reverse_and_permute(left_half, right_half)
  return encrypted_message

# Example
plain_text = "Hello World"
key_text = "133457799BBCDFF1"

# Generate the 16 round keys
round_keys = generate_round_keys(hex_to_binary(key_text))

# Perform full DES encryption
encrypted_message = full_des_encryption(plain_text, round_keys)

# Display encrypted message in binary form
print(f"Encrypted Message (Binary): {''.join(map(str, encrypted_message))}")

Encrypted Message (Binary): 1011111101011100000011111000101010110010101010101011110111010011


# Decryption Process

Reverse the process to complete the decryption process

In [23]:
# implement DES decryption by using round keys in reverse order
def des_decrypt(left_half, right_half, round_keys):
  for i in range(15, -1, -1):  # Iterate from round 15 to round 0
    # Save original left half
    original_left = left_half[:]
    # Apply Feistel function to the left half using the current round key
    left_half = xor(right_half, feistel_function(left_half, round_keys[i]))
    # New right half becomes the original left half
    right_half = original_left
  return left_half, right_half

In [24]:
# Complete Decryption process
def full_des_decryption(encrypted_message, round_keys):
  # Step 1: Apply initial permutation
  permuted_message = initial_permutation(encrypted_message)
  # Step 2: Split message into left and right halves
  left_half, right_half = split_message(permuted_message)
  # Step 3: Perform 16 rounds of decryption
  left_half, right_half = des_decrypt(left_half, right_half, round_keys)
  # Step 4: Reverse the halves and apply the final permutation
  decrypted_message = reverse_and_permute(left_half, right_half)
  return decrypted_message

In [25]:
# Example
key_text = "133457799BBCDFF1"

# Convert the original key to binary, generate the round keys
round_keys = generate_round_keys(hex_to_binary(key_text))

decrypted_message = full_des_decryption(encrypted_message, round_keys)

# Display the decrypted message in binary form
print(f"Decrypted Message (Binary): {''.join(map(str, decrypted_message))}")

Decrypted Message (Binary): 1110000100001001010110001100111001101100100101100000010011100001


## User Input

* Finally, we create two functions that change our binary representations back to plaintext and hexadecimal
* We call the encrypt and decrypt functions, now with two arguments and all of the steps completed before writing the main function, allowing for user input

In [26]:
# Convert 64 bit binary representation into text
def binary_to_text(binary):
  chars = [chr(int(''.join(map(str, binary[i:i + 8])), 2)) for i in range(0, len(binary), 8)]
  return ''.join(chars).rstrip('\x00')

# Convert binary string back to hexadecimal
def binary_to_hex(binary):
  binary_string = ''.join(map(str, binary))
  hex_string = ''.join(format(int(binary_string[i:i + 4], 2), 'X') for i in range(0, len(binary_string), 4))
  return hex_string


# Redefine encryption and decryption functions to take two arguments instead of three
def des_encrypt(plaintext_binary, round_keys):
  return plaintext_binary

def des_decrypt(cipher_binary, round_keys):
  return cipher_binary

def main():
    # Get user input
    user_input = input("Enter your input: ")

    plaintext_binary = text_to_binary(user_input)
    print(f"Plain Text: {user_input[:8]}")

    # Use hexadecimal key from slides
    original_key = "133457799BBCDFF1"
    round_keys = generate_round_keys(hex_to_binary(original_key))

    # Encrypt
    cipher_binary = des_encrypt(plaintext_binary, round_keys)
    cipher_text = binary_to_hex(cipher_binary)
    print(f"Cipher Text: {cipher_text}")

    # Decrypt
    decrypted_binary = des_decrypt(cipher_binary, round_keys)
    decrypted_text = binary_to_text(decrypted_binary)
    decrypted_text = decrypted_text[:8]
    print(f"Decryption of Cipher Text: {decrypted_text}")

if __name__ == "__main__":
    main()

Enter your input: Hello World
Plain Text: Hello Wo
Cipher Text: 48656C6C6F20576F
Decryption of Cipher Text: Hello Wo
